In [38]:
!pip install --quiet tensorflow-text
# Clone the entire repo.
%cd /content/
!rm -r -f dl4tm
!git clone git://github.com/Jeilef/DL4TM_Text2Python.git dl4tm
%cd dl4tm
!ls

/content
Cloning into 'dl4tm'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 79 (delta 37), reused 39 (delta 13), pack-reused 0
Receiving objects: 100% (79/79), 395.16 KiB | 6.48 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/dl4tm
conala-corpus-v1.1  README.md	      text2python.ipynb  transformer_util.py
google.ipynb	    requirements.txt  transformer.ipynb


In [39]:
import numpy as np
import json
import logging
from tokenize import tokenize
from io import BytesIO
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_text # See https://github.com/tensorflow/hub/issues/463

In [40]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [41]:
train_file = open("conala-corpus-v1.1/conala-corpus/conala-train.json", "r")
train_json = json.load(train_file)

print(len(train_json))
print(train_json[0])

2379
{'intent': 'How to convert a list of multiple integers into a single integer?', 'rewritten_intent': "Concatenate elements of a list 'x' of multiple integers to a single integer", 'snippet': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))', 'question_id': 41067960}


In [42]:
base_sentences = np.array([item["rewritten_intent"] for item in train_json])
codes = np.array([item["snippet"] for item in train_json])

print(base_sentences[0])
print(codes[0])

Concatenate elements of a list 'x' of multiple integers to a single integer
sum(d * 10 ** i for i, d in enumerate(x[::-1]))


In [43]:
# Use custom tokenize function for code snippets
def tokenize_python(s):
    tokenized = tokenize(BytesIO(s.numpy()).readline)
    tokens = [token[1] for token in tokenized]
    tensor = tf.convert_to_tensor(tokens, dtype=tf.string)
    
    return tensor

example = tf.constant(codes[0])
tokenize_python(example)

<tf.Tensor: shape=(26,), dtype=string, numpy=
array([b'utf-8', b'sum', b'(', b'd', b'*', b'10', b'**', b'i', b'for',
       b'i', b',', b'd', b'in', b'enumerate', b'(', b'x', b'[', b':',
       b':', b'-', b'1', b']', b')', b')', b'', b''], dtype=object)>

In [44]:
data = tf.data.Dataset.from_tensor_slices(codes)
data = data.map(lambda x : tf.py_function(func=tokenize_python, inp=[x], Tout=tf.string))

In [45]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask

def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]


class DecoderMasking(tf.keras.layers.Layer):
  def call(self, targets):
    padded_targets = create_padding_mask(targets)
    look_ahead_mask = create_look_ahead_mask(tf.shape(targets)[1])
    return tf.maximum(padded_targets, look_ahead_mask)

class DecoderPaddingMasked(tf.keras.layers.Layer):
  def call(self, inputs):
    expanded_inputs = inputs[:, tf.newaxis, tf.newaxis, :]
    return tf.cast(expanded_inputs, tf.float32)

example = tf.constant(codes[0])
tokenized_example = tokenize_python(example)

vectorize_layer = keras.layers.experimental.preprocessing.StringLookup(max_tokens=8000)
vectorize_layer.adapt(data)

vectorized_example = vectorize_layer(tokenized_example)

masking = DecoderMasking()
lam = masking(tf.convert_to_tensor(([vectorized_example])))
print(lam.shape)

(1, 1, 26, 26)


In [46]:
# Pretrained BERT encoder
encoder_inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [47]:
# Transformer Decoder  taken from https://www.tensorflow.org/tutorials/text/transformer
from transformer_util import Decoder
output_tokens = 8000
sample_decoder = Decoder(num_layers=2, d_model=128, num_heads=2,
                         dff=128, target_vocab_size=output_tokens,
                         maximum_position_encoding=5000)

decoder_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)

# Apply masking layer here
look_ahead_mask = DecoderMasking()(decoder_input)
dec_padding_mask = DecoderPaddingMasked()(encoder_inputs['input_mask'])

decoder_output, attn = sample_decoder(decoder_input,
                              enc_output=sequence_output,
                              training=False,
                              look_ahead_mask=look_ahead_mask,
                              padding_mask=dec_padding_mask)
final_layer = tf.keras.layers.Dense(output_tokens)
output = final_layer(decoder_output)


embedding_model = tf.keras.Model((text_input, decoder_input), output)
embedding_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
keras_layer_6 (KerasLayer)      {'input_word_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_7 (KerasLayer)      {'encoder_outputs':  4385921     keras_layer_6[0][0]              
                                                                 keras_layer_6[0][1]        

In [49]:
sentence = tf.constant([train_json[0]["rewritten_intent"]])
snippet = tf.constant(train_json[0]["snippet"])
snippet = tf.py_function(func=tokenize_python, inp=[snippet], Tout=tf.string)
snippet = tf.expand_dims(snippet, axis=0)
print(sentence.shape, snippet.shape)
# result = embedding_model((sentence, snippet))
result

(1,) (1, 26)


<tf.Tensor: shape=(1, 26, 8000), dtype=float32, numpy=
array([[[ 0.25900948,  0.05902249,  0.00255737, ..., -0.14142132,
          0.060927  ,  0.23605488],
        [ 0.2641915 ,  0.04997048,  0.11746741, ..., -0.11563613,
          0.0299585 ,  0.2452411 ],
        [ 0.22051044, -0.03371699,  0.07480508, ..., -0.12167043,
          0.09683892,  0.18546578],
        ...,
        [ 0.21927257, -0.07914125,  0.10151628, ..., -0.08671107,
          0.06166898,  0.28820792],
        [ 0.1649797 , -0.09419025,  0.05277481, ..., -0.0641139 ,
          0.06363688,  0.22325364],
        [ 0.16321684, -0.08993292,  0.04195784, ..., -0.05346364,
          0.04414991,  0.2221666 ]]], dtype=float32)>

In [50]:
# TODO: Preprocess code samples
# TODO: Set up masking
# TODO: Run training

In [51]:
# Taken from https://www.tensorflow.org/tutorials/text/transformer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                     beta_2=0.98, epsilon=1e-9)
embedding_model.compile(optimizer, loss_function)

In [52]:
tokenized_codes = [tf.py_function(func=tokenize_python, inp=[snippet], Tout=tf.string) for snippet in codes]
vectorize_layer = keras.layers.experimental.preprocessing.StringLookup(max_tokens=output_tokens)
vectorize_layer.adapt(data)
vectorized_inputs = [vectorize_layer(code) for code in tokenized_codes]
tokenized_codes = tf.keras.preprocessing.sequence.pad_sequences(vectorized_inputs, value=0)


In [53]:
base_sentences = np.array(['' if v is None else v for v in base_sentences])

In [ ]:
history = embedding_model.fit([base_sentences, tokenized_codes], tokenized_codes, batch_size=64, epochs=10)

Epoch 1/10
38/38 [==============================] - 94s 2s/step - loss: 7.4011
Epoch 2/10
38/38 [==============================] - 84s 2s/step - loss: 3.3615
Epoch 3/10
38/38 [==============================] - 83s 2s/step - loss: 1.5874
Epoch 4/10
38/38 [==============================] - 83s 2s/step - loss: 0.9649
Epoch 5/10
38/38 [==============================] - 83s 2s/step - loss: 0.6680
Epoch 6/10
38/38 [==============================] - 83s 2s/step - loss: 0.4806
Epoch 7/10
38/38 [==============================] - 83s 2s/step - loss: 0.3630
Epoch 8/10
18/38 [=============>................] - ETA: 48s - loss: 0.2566

In [ ]:
def generate_result(test_sentence, model):
  prediction = ""
  while len(prediction) < 232:
    prediction += model.predict(([test_sentence], [tokenize_python(prediction)]))
  return prediction

In [ ]:
print(generate_result("add 5 and 6."))